# SOLUTIONS: Advanced ML Week 1, Lecture 1: Working with and Preparing Text Data

In this notebook we will be preparing Twitter (X) Tweets for sentiment analysis.  Sentiment analysis is a common text classification challenge to determine whether a text is positive or negative.  

This is useful for companies that want to analyze large numbers of documents, tweets, reviews, etc., to determine public sentiment about a product or service.

The data was originally gathered from Twitter (now X) and hand-labeled.  Of course there will be some human bias in the labeling.  It was downloaded from Kaggle at this site: [Kaggle Twitter Tweets Sentiment Dataset](https://www.kaggle.com/datasets/yasserh/twitter-tweets-sentiment-dataset/)

There are 3 classes: positive, negative, and neutral.

In [1]:
## Import necessary packages
import pandas as pd
import nltk

# Load the Data

We will download our **corpus** of tweets.

In [2]:
## Download corpus of tweets
df = pd.read_csv('../Data/archive.zip')
df.head(10)

,textID,text,selected_text,sentiment
0,cb774db0d1,"I`d have responded, if I were going","I`d have responded, if I were going",neutral
1,549e992a42,Sooo SAD I will miss you here in San Diego!!!,Sooo SAD,negative
2,088c60f138,my boss is bullying me...,bullying me,negative
3,9642c003ef,what interview! leave me alone,leave me alone,negative
4,358bd9e861,"Sons of ****, why couldn`t they put them on t...","Sons of ****,",negative
5,28b57f3990,http://www.dothebouncy.com/smf - some shameles...,http://www.dothebouncy.com/smf - some shameles...,neutral
6,6e0c6d75b1,2am feedings for the baby are fun when he is a...,fun,positive
7,50e14c0bb8,Soooo high,Soooo high,neutral
8,e050245fbd,Both of you,Both of you,neutral
9,fc2cbefa9d,Journey!? Wow... u just became cooler. hehe....,Wow... u just became cooler.,positive


# Some light EDA

In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 27481 entries, 0 to 27480
Data columns (total 4 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   textID         27481 non-null  object
 1   text           27480 non-null  object
 2   selected_text  27480 non-null  object
 3   sentiment      27481 non-null  object
dtypes: object(4)
memory usage: 858.9+ KB


In [4]:
df.duplicated().sum()

0

# Some Light Data Cleaning

We see that our **corpus** has 27481 **documents**, each with an ID, the full text, a shortened version, and the labeled sentiment.

Interestingly, one of the tweets has no text!  We definitely want to get rid of that.  We will also drop the `textID` and `selected_text` columns.  We are going to use the entire text of each tweet, not just a subset.

We will keep the label, `sentiment` for later classification and analysis tasks.

In [5]:
df = df.drop(columns=['textID', 'selected_text'])
df = df.dropna()

In [6]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 27480 entries, 0 to 27480
Data columns (total 2 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   text       27480 non-null  object
 1   sentiment  27480 non-null  object
dtypes: object(2)
memory usage: 644.1+ KB


# Some More EDA
Let's look at some aspects of this text.
* What do the **documents** look like?
* How long do the tend to be?

## View some sample tweets

In [7]:
## Expand how many characters pandas will show
pd.set_option('display.max_colwidth', None)

## Display some of the documents (tweets)
df['text'].head(10)

0                                                             I`d have responded, if I were going
1                                                   Sooo SAD I will miss you here in San Diego!!!
2                                                                       my boss is bullying me...
3                                                                  what interview! leave me alone
4                      Sons of ****, why couldn`t they put them on the releases we already bought
5    http://www.dothebouncy.com/smf - some shameless plugging for the best Rangers forum on earth
6                                2am feedings for the baby are fun when he is all smiles and coos
7                                                                                      Soooo high
8                                                                                     Both of you
9                            Journey!? Wow... u just became cooler.  hehe... (is that possible!?)
Name: text, dtype: o

We see a URL in here.  This is a problem for normalization, since those are going to be unique and not covered by stop words

## URLs

In [8]:
df.loc[df['text'].str.contains('http://')]

,text,sentiment
5,http://www.dothebouncy.com/smf - some shameless plugging for the best Rangers forum on earth,neutral
17,"i`ve been sick for the past few days and thus, my hair looks wierd. if i didnt have a hat on it would look... http://tinyurl.com/mnf4kw",negative
35,"Thats it, its the end. Tears for Fears vs Eric Prydz, DJ Hero http://bit.ly/2Hpbg4",neutral
50,Then you should check out http://twittersucks.com and connect with other tweeple who hate twitter,neutral
57,will be back later. http://plurk.com/p/rp3k7,neutral
...,...,...
27374,"says Finally, Im home. http://plurk.com/p/rr121",neutral
27384,This is a much better tool than some I have come across http://www.tweepular.com - Twitter Karma on Steroids,positive
27386,#vwll2009 Would one of the VWLLers want to add this event to our Ning? http://bit.ly/BF5sh Would much appreciate that,positive
27463,LIKE DREW SAID 'GIVE TC A CHANCE' WE WILL MISS THOMAS BUT HAVE TO MOVE ON. SO WATCH THIS! http://bit.ly/r6RfC,negative


## Get some statistics on the length of **documents**

In [9]:
## Determine the length of each tweet
df['length'] = df['text'].map(len)
df.head(10)

,text,sentiment,length
0,"I`d have responded, if I were going",neutral,36
1,Sooo SAD I will miss you here in San Diego!!!,negative,46
2,my boss is bullying me...,negative,25
3,what interview! leave me alone,negative,31
4,"Sons of ****, why couldn`t they put them on the releases we already bought",negative,75
5,http://www.dothebouncy.com/smf - some shameless plugging for the best Rangers forum on earth,neutral,92
6,2am feedings for the baby are fun when he is all smiles and coos,positive,64
7,Soooo high,neutral,10
8,Both of you,neutral,12
9,Journey!? Wow... u just became cooler. hehe... (is that possible!?),positive,69


In [10]:
## Analyze the statistics of the lengths
df['length'].describe()

count    27480.000000
mean        68.330022
std         35.603870
min          3.000000
25%         39.000000
50%         64.000000
75%         97.000000
max        141.000000
Name: length, dtype: float64

The tweets have an mean length of 68 characters and a median of 64. They range from 3 to 141 characters with a standard deviation of 35.  The middle 50% are between 39 and 97 characters in length.

This gives us some idea of how long they tend to be.

# Text Normalization with NLTK

## Normalizing Casing

It's common practice to lower the casing of the text in our documents to contribut to normalizing.

In [11]:
df['lower_text'] = df['text'].str.lower()
df.head()

,text,sentiment,length,lower_text
0,"I`d have responded, if I were going",neutral,36,"i`d have responded, if i were going"
1,Sooo SAD I will miss you here in San Diego!!!,negative,46,sooo sad i will miss you here in san diego!!!
2,my boss is bullying me...,negative,25,my boss is bullying me...
3,what interview! leave me alone,negative,31,what interview! leave me alone
4,"Sons of ****, why couldn`t they put them on the releases we already bought",negative,75,"sons of ****, why couldn`t they put them on the releases we already bought"


## Tokenizing

Tokenizing text into single word tokens is simple in Python.  We can just use `str.split()`.  The default separator for `.split()` is one space, so `' '`.

We can access Pandas' string accessor with `df.str.<method>`.  This allows us to apply string methods to all rows in a column.

When processing text, if memory allows, it can be useful to keep many versions of your text: tokenize, lemmatized, no stop words, etc.  Some analysis or modeling packages expect tokenized data and others do not.  We often want to use different versions for different kinds of analysis, too.

In [12]:
df['tokens'] = df['lower_text'].str.split()
df.head()

,text,sentiment,length,lower_text,tokens
0,"I`d have responded, if I were going",neutral,36,"i`d have responded, if i were going","[i`d, have, responded,, if, i, were, going]"
1,Sooo SAD I will miss you here in San Diego!!!,negative,46,sooo sad i will miss you here in san diego!!!,"[sooo, sad, i, will, miss, you, here, in, san, diego!!!]"
2,my boss is bullying me...,negative,25,my boss is bullying me...,"[my, boss, is, bullying, me...]"
3,what interview! leave me alone,negative,31,what interview! leave me alone,"[what, interview!, leave, me, alone]"
4,"Sons of ****, why couldn`t they put them on the releases we already bought",negative,75,"sons of ****, why couldn`t they put them on the releases we already bought","[sons, of, ****,, why, couldn`t, they, put, them, on, the, releases, we, already, bought]"


### Better way to tokenize data

NLTK has a more sophisticated tokenization function that will isolate things like punctuation as well.  This way 'hooray' and 'hooray!!!' will be the same token.

In order for NLTK to recognize the punctuation, we will need to download the 'punkt' data.

In [13]:
## Download punkt
nltk.download('punkt')

df['tokens'] = df['lower_text'].apply(nltk.word_tokenize)
df.head()

[nltk_data] Downloading package punkt to
[nltk_data]     /Users/codingdojo/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


,text,sentiment,length,lower_text,tokens
0,"I`d have responded, if I were going",neutral,36,"i`d have responded, if i were going","[i, `, d, have, responded, ,, if, i, were, going]"
1,Sooo SAD I will miss you here in San Diego!!!,negative,46,sooo sad i will miss you here in san diego!!!,"[sooo, sad, i, will, miss, you, here, in, san, diego, !, !, !]"
2,my boss is bullying me...,negative,25,my boss is bullying me...,"[my, boss, is, bullying, me, ...]"
3,what interview! leave me alone,negative,31,what interview! leave me alone,"[what, interview, !, leave, me, alone]"
4,"Sons of ****, why couldn`t they put them on the releases we already bought",negative,75,"sons of ****, why couldn`t they put them on the releases we already bought","[sons, of, *, *, *, *, ,, why, couldn, `, t, they, put, them, on, the, releases, we, already, bought]"


## Remove Stop Words

In [14]:
## Download NLTK stopword list
nltk.download('stopwords')

## Load the English stop words.
stop_words = nltk.corpus.stopwords.words('english')
stop_words[:10]

[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/codingdojo/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


['i', 'me', 'my', 'myself', 'we', 'our', 'ours', 'ourselves', 'you', "you're"]

<font color=red> NOTICE </font> that all of the stop words are lower case.  It's necessary to ensure that your tokens are all lower case before using this list to remove stop words.

To remove the stop words from each document, we will apply a function that will check each word in the list of tokens against the list of stopwords and remove them if they are in the list.  More specifically, it will only save them if they are NOT in the list.

In [15]:
## [V1 List Comp] Remove Stopwords Function
def remove_stopwords(tokens):
    no_stops = [token for token in tokens if not token in stop_words]
    return no_stops

df['no_stops'] = df['tokens'].map(remove_stopwords)
df.head()

,text,sentiment,length,lower_text,tokens,no_stops
0,"I`d have responded, if I were going",neutral,36,"i`d have responded, if i were going","[i, `, d, have, responded, ,, if, i, were, going]","[`, responded, ,, going]"
1,Sooo SAD I will miss you here in San Diego!!!,negative,46,sooo sad i will miss you here in san diego!!!,"[sooo, sad, i, will, miss, you, here, in, san, diego, !, !, !]","[sooo, sad, miss, san, diego, !, !, !]"
2,my boss is bullying me...,negative,25,my boss is bullying me...,"[my, boss, is, bullying, me, ...]","[boss, bullying, ...]"
3,what interview! leave me alone,negative,31,what interview! leave me alone,"[what, interview, !, leave, me, alone]","[interview, !, leave, alone]"
4,"Sons of ****, why couldn`t they put them on the releases we already bought",negative,75,"sons of ****, why couldn`t they put them on the releases we already bought","[sons, of, *, *, *, *, ,, why, couldn, `, t, they, put, them, on, the, releases, we, already, bought]","[sons, *, *, *, *, ,, `, put, releases, already, bought]"


In [16]:
## [V2 For Loop] Remove Stopwords Function
def remove_stopwords(tokens):
    no_stops = []
    for token in tokens:
        if token not in stop_words:
            no_stops.append(token)            
    return no_stops

df['no_stops'] = df['tokens'].map(remove_stopwords)
df.head(10)

,text,sentiment,length,lower_text,tokens,no_stops
0,"I`d have responded, if I were going",neutral,36,"i`d have responded, if i were going","[i, `, d, have, responded, ,, if, i, were, going]","[`, responded, ,, going]"
1,Sooo SAD I will miss you here in San Diego!!!,negative,46,sooo sad i will miss you here in san diego!!!,"[sooo, sad, i, will, miss, you, here, in, san, diego, !, !, !]","[sooo, sad, miss, san, diego, !, !, !]"
2,my boss is bullying me...,negative,25,my boss is bullying me...,"[my, boss, is, bullying, me, ...]","[boss, bullying, ...]"
3,what interview! leave me alone,negative,31,what interview! leave me alone,"[what, interview, !, leave, me, alone]","[interview, !, leave, alone]"
4,"Sons of ****, why couldn`t they put them on the releases we already bought",negative,75,"sons of ****, why couldn`t they put them on the releases we already bought","[sons, of, *, *, *, *, ,, why, couldn, `, t, they, put, them, on, the, releases, we, already, bought]","[sons, *, *, *, *, ,, `, put, releases, already, bought]"
5,http://www.dothebouncy.com/smf - some shameless plugging for the best Rangers forum on earth,neutral,92,http://www.dothebouncy.com/smf - some shameless plugging for the best rangers forum on earth,"[http, :, //www.dothebouncy.com/smf, -, some, shameless, plugging, for, the, best, rangers, forum, on, earth]","[http, :, //www.dothebouncy.com/smf, -, shameless, plugging, best, rangers, forum, earth]"
6,2am feedings for the baby are fun when he is all smiles and coos,positive,64,2am feedings for the baby are fun when he is all smiles and coos,"[2am, feedings, for, the, baby, are, fun, when, he, is, all, smiles, and, coos]","[2am, feedings, baby, fun, smiles, coos]"
7,Soooo high,neutral,10,soooo high,"[soooo, high]","[soooo, high]"
8,Both of you,neutral,12,both of you,"[both, of, you]",[]
9,Journey!? Wow... u just became cooler. hehe... (is that possible!?),positive,69,journey!? wow... u just became cooler. hehe... (is that possible!?),"[journey, !, ?, wow, ..., u, just, became, cooler, ., hehe, ..., (, is, that, possible, !, ?, )]","[journey, !, ?, wow, ..., u, became, cooler, ., hehe, ..., (, possible, !, ?, )]"


In [17]:
## [v3 For Loop - Using continue] Remove Stopwords Function
def remove_stopwords(tokens):
    no_stops = []
    
    for token in tokens:
        if token in stop_words:
            continue
            
        else:
            no_stops.append(token)
            
    return no_stops

df['no_stops'] = df['tokens'].map(remove_stopwords)
df.head(10)

,text,sentiment,length,lower_text,tokens,no_stops
0,"I`d have responded, if I were going",neutral,36,"i`d have responded, if i were going","[i, `, d, have, responded, ,, if, i, were, going]","[`, responded, ,, going]"
1,Sooo SAD I will miss you here in San Diego!!!,negative,46,sooo sad i will miss you here in san diego!!!,"[sooo, sad, i, will, miss, you, here, in, san, diego, !, !, !]","[sooo, sad, miss, san, diego, !, !, !]"
2,my boss is bullying me...,negative,25,my boss is bullying me...,"[my, boss, is, bullying, me, ...]","[boss, bullying, ...]"
3,what interview! leave me alone,negative,31,what interview! leave me alone,"[what, interview, !, leave, me, alone]","[interview, !, leave, alone]"
4,"Sons of ****, why couldn`t they put them on the releases we already bought",negative,75,"sons of ****, why couldn`t they put them on the releases we already bought","[sons, of, *, *, *, *, ,, why, couldn, `, t, they, put, them, on, the, releases, we, already, bought]","[sons, *, *, *, *, ,, `, put, releases, already, bought]"
5,http://www.dothebouncy.com/smf - some shameless plugging for the best Rangers forum on earth,neutral,92,http://www.dothebouncy.com/smf - some shameless plugging for the best rangers forum on earth,"[http, :, //www.dothebouncy.com/smf, -, some, shameless, plugging, for, the, best, rangers, forum, on, earth]","[http, :, //www.dothebouncy.com/smf, -, shameless, plugging, best, rangers, forum, earth]"
6,2am feedings for the baby are fun when he is all smiles and coos,positive,64,2am feedings for the baby are fun when he is all smiles and coos,"[2am, feedings, for, the, baby, are, fun, when, he, is, all, smiles, and, coos]","[2am, feedings, baby, fun, smiles, coos]"
7,Soooo high,neutral,10,soooo high,"[soooo, high]","[soooo, high]"
8,Both of you,neutral,12,both of you,"[both, of, you]",[]
9,Journey!? Wow... u just became cooler. hehe... (is that possible!?),positive,69,journey!? wow... u just became cooler. hehe... (is that possible!?),"[journey, !, ?, wow, ..., u, just, became, cooler, ., hehe, ..., (, is, that, possible, !, ?, )]","[journey, !, ?, wow, ..., u, became, cooler, ., hehe, ..., (, possible, !, ?, )]"


## Remove Punctuation

We can remove punctuation in a similar that we removed stop words.  However, we will get our list of punctuation from the built in Python string library.

In [18]:
## Import built-in String Libary
from string import punctuation
print(punctuation)

!"#$%&'()*+,-./:;<=>?@[\]^_`{|}~


In [19]:
## [v1 List Comp] Remove punctuation
def remove_punct(tokens):
    no_punct = [token for token in tokens if not token in punctuation]
    return no_punct

df['no_stops_no_punct'] = df['no_stops'].apply(remove_punct)
df.head(10)

,text,sentiment,length,lower_text,tokens,no_stops,no_stops_no_punct
0,"I`d have responded, if I were going",neutral,36,"i`d have responded, if i were going","[i, `, d, have, responded, ,, if, i, were, going]","[`, responded, ,, going]","[responded, going]"
1,Sooo SAD I will miss you here in San Diego!!!,negative,46,sooo sad i will miss you here in san diego!!!,"[sooo, sad, i, will, miss, you, here, in, san, diego, !, !, !]","[sooo, sad, miss, san, diego, !, !, !]","[sooo, sad, miss, san, diego]"
2,my boss is bullying me...,negative,25,my boss is bullying me...,"[my, boss, is, bullying, me, ...]","[boss, bullying, ...]","[boss, bullying, ...]"
3,what interview! leave me alone,negative,31,what interview! leave me alone,"[what, interview, !, leave, me, alone]","[interview, !, leave, alone]","[interview, leave, alone]"
4,"Sons of ****, why couldn`t they put them on the releases we already bought",negative,75,"sons of ****, why couldn`t they put them on the releases we already bought","[sons, of, *, *, *, *, ,, why, couldn, `, t, they, put, them, on, the, releases, we, already, bought]","[sons, *, *, *, *, ,, `, put, releases, already, bought]","[sons, put, releases, already, bought]"
5,http://www.dothebouncy.com/smf - some shameless plugging for the best Rangers forum on earth,neutral,92,http://www.dothebouncy.com/smf - some shameless plugging for the best rangers forum on earth,"[http, :, //www.dothebouncy.com/smf, -, some, shameless, plugging, for, the, best, rangers, forum, on, earth]","[http, :, //www.dothebouncy.com/smf, -, shameless, plugging, best, rangers, forum, earth]","[http, //www.dothebouncy.com/smf, shameless, plugging, best, rangers, forum, earth]"
6,2am feedings for the baby are fun when he is all smiles and coos,positive,64,2am feedings for the baby are fun when he is all smiles and coos,"[2am, feedings, for, the, baby, are, fun, when, he, is, all, smiles, and, coos]","[2am, feedings, baby, fun, smiles, coos]","[2am, feedings, baby, fun, smiles, coos]"
7,Soooo high,neutral,10,soooo high,"[soooo, high]","[soooo, high]","[soooo, high]"
8,Both of you,neutral,12,both of you,"[both, of, you]",[],[]
9,Journey!? Wow... u just became cooler. hehe... (is that possible!?),positive,69,journey!? wow... u just became cooler. hehe... (is that possible!?),"[journey, !, ?, wow, ..., u, just, became, cooler, ., hehe, ..., (, is, that, possible, !, ?, )]","[journey, !, ?, wow, ..., u, became, cooler, ., hehe, ..., (, possible, !, ?, )]","[journey, wow, ..., u, became, cooler, hehe, ..., possible]"


In [20]:
# [v2 For Loop] Remove punctuation 
def remove_punct(tokens):
    no_punct = []
    for token in tokens:
        if token not in punctuation:
            no_punct.append(token)
    return no_punct

df['no_stops_no_punct'] = df['no_stops'].apply(remove_punct)
df.head(10)

,text,sentiment,length,lower_text,tokens,no_stops,no_stops_no_punct
0,"I`d have responded, if I were going",neutral,36,"i`d have responded, if i were going","[i, `, d, have, responded, ,, if, i, were, going]","[`, responded, ,, going]","[responded, going]"
1,Sooo SAD I will miss you here in San Diego!!!,negative,46,sooo sad i will miss you here in san diego!!!,"[sooo, sad, i, will, miss, you, here, in, san, diego, !, !, !]","[sooo, sad, miss, san, diego, !, !, !]","[sooo, sad, miss, san, diego]"
2,my boss is bullying me...,negative,25,my boss is bullying me...,"[my, boss, is, bullying, me, ...]","[boss, bullying, ...]","[boss, bullying, ...]"
3,what interview! leave me alone,negative,31,what interview! leave me alone,"[what, interview, !, leave, me, alone]","[interview, !, leave, alone]","[interview, leave, alone]"
4,"Sons of ****, why couldn`t they put them on the releases we already bought",negative,75,"sons of ****, why couldn`t they put them on the releases we already bought","[sons, of, *, *, *, *, ,, why, couldn, `, t, they, put, them, on, the, releases, we, already, bought]","[sons, *, *, *, *, ,, `, put, releases, already, bought]","[sons, put, releases, already, bought]"
5,http://www.dothebouncy.com/smf - some shameless plugging for the best Rangers forum on earth,neutral,92,http://www.dothebouncy.com/smf - some shameless plugging for the best rangers forum on earth,"[http, :, //www.dothebouncy.com/smf, -, some, shameless, plugging, for, the, best, rangers, forum, on, earth]","[http, :, //www.dothebouncy.com/smf, -, shameless, plugging, best, rangers, forum, earth]","[http, //www.dothebouncy.com/smf, shameless, plugging, best, rangers, forum, earth]"
6,2am feedings for the baby are fun when he is all smiles and coos,positive,64,2am feedings for the baby are fun when he is all smiles and coos,"[2am, feedings, for, the, baby, are, fun, when, he, is, all, smiles, and, coos]","[2am, feedings, baby, fun, smiles, coos]","[2am, feedings, baby, fun, smiles, coos]"
7,Soooo high,neutral,10,soooo high,"[soooo, high]","[soooo, high]","[soooo, high]"
8,Both of you,neutral,12,both of you,"[both, of, you]",[],[]
9,Journey!? Wow... u just became cooler. hehe... (is that possible!?),positive,69,journey!? wow... u just became cooler. hehe... (is that possible!?),"[journey, !, ?, wow, ..., u, just, became, cooler, ., hehe, ..., (, is, that, possible, !, ?, )]","[journey, !, ?, wow, ..., u, became, cooler, ., hehe, ..., (, possible, !, ?, )]","[journey, wow, ..., u, became, cooler, hehe, ..., possible]"


In [21]:
# [v3 For Loop - Continue] Remove punctuation 
def remove_punct(tokens):
    no_punct = []
    for token in tokens:
        if token in punctuation:
            continue
        else:
            no_punct.append(token)
    return no_punct

df['no_stops_no_punct'] = df['no_stops'].apply(remove_punct)
df.head(10)

,text,sentiment,length,lower_text,tokens,no_stops,no_stops_no_punct
0,"I`d have responded, if I were going",neutral,36,"i`d have responded, if i were going","[i, `, d, have, responded, ,, if, i, were, going]","[`, responded, ,, going]","[responded, going]"
1,Sooo SAD I will miss you here in San Diego!!!,negative,46,sooo sad i will miss you here in san diego!!!,"[sooo, sad, i, will, miss, you, here, in, san, diego, !, !, !]","[sooo, sad, miss, san, diego, !, !, !]","[sooo, sad, miss, san, diego]"
2,my boss is bullying me...,negative,25,my boss is bullying me...,"[my, boss, is, bullying, me, ...]","[boss, bullying, ...]","[boss, bullying, ...]"
3,what interview! leave me alone,negative,31,what interview! leave me alone,"[what, interview, !, leave, me, alone]","[interview, !, leave, alone]","[interview, leave, alone]"
4,"Sons of ****, why couldn`t they put them on the releases we already bought",negative,75,"sons of ****, why couldn`t they put them on the releases we already bought","[sons, of, *, *, *, *, ,, why, couldn, `, t, they, put, them, on, the, releases, we, already, bought]","[sons, *, *, *, *, ,, `, put, releases, already, bought]","[sons, put, releases, already, bought]"
5,http://www.dothebouncy.com/smf - some shameless plugging for the best Rangers forum on earth,neutral,92,http://www.dothebouncy.com/smf - some shameless plugging for the best rangers forum on earth,"[http, :, //www.dothebouncy.com/smf, -, some, shameless, plugging, for, the, best, rangers, forum, on, earth]","[http, :, //www.dothebouncy.com/smf, -, shameless, plugging, best, rangers, forum, earth]","[http, //www.dothebouncy.com/smf, shameless, plugging, best, rangers, forum, earth]"
6,2am feedings for the baby are fun when he is all smiles and coos,positive,64,2am feedings for the baby are fun when he is all smiles and coos,"[2am, feedings, for, the, baby, are, fun, when, he, is, all, smiles, and, coos]","[2am, feedings, baby, fun, smiles, coos]","[2am, feedings, baby, fun, smiles, coos]"
7,Soooo high,neutral,10,soooo high,"[soooo, high]","[soooo, high]","[soooo, high]"
8,Both of you,neutral,12,both of you,"[both, of, you]",[],[]
9,Journey!? Wow... u just became cooler. hehe... (is that possible!?),positive,69,journey!? wow... u just became cooler. hehe... (is that possible!?),"[journey, !, ?, wow, ..., u, just, became, cooler, ., hehe, ..., (, is, that, possible, !, ?, )]","[journey, !, ?, wow, ..., u, became, cooler, ., hehe, ..., (, possible, !, ?, )]","[journey, wow, ..., u, became, cooler, hehe, ..., possible]"


## Remove URLs

We can see that NLTK divided the 'http' and '//www.' portions of the web addresses.  We will need to deal with those separately.

In [22]:
## [v1 List Comp] Define function to remove URLs
def remove_urls(token_list):
    no_urls = [token for token in token_list if not 'http' in token and not 'www' in token]
    return no_urls

## Remove URLs from no_stops_no_punct
df['no_stops_no_punct'] = df['no_stops_no_punct'].apply(remove_urls)

In [23]:
## [v2 For Loop] Define function to remove URLs
def remove_urls(token_list):
    no_urls = []
    for token in token_list:
        if ('http' not in token) & ('www' not in token):
            no_urls.append(token)
    return no_urls

## Remove URLs from no_stops_no_punct
df['no_stops_no_punct'] = df['no_stops_no_punct'].apply(remove_urls)
df

,text,sentiment,length,lower_text,tokens,no_stops,no_stops_no_punct
0,"I`d have responded, if I were going",neutral,36,"i`d have responded, if i were going","[i, `, d, have, responded, ,, if, i, were, going]","[`, responded, ,, going]","[responded, going]"
1,Sooo SAD I will miss you here in San Diego!!!,negative,46,sooo sad i will miss you here in san diego!!!,"[sooo, sad, i, will, miss, you, here, in, san, diego, !, !, !]","[sooo, sad, miss, san, diego, !, !, !]","[sooo, sad, miss, san, diego]"
2,my boss is bullying me...,negative,25,my boss is bullying me...,"[my, boss, is, bullying, me, ...]","[boss, bullying, ...]","[boss, bullying, ...]"
3,what interview! leave me alone,negative,31,what interview! leave me alone,"[what, interview, !, leave, me, alone]","[interview, !, leave, alone]","[interview, leave, alone]"
4,"Sons of ****, why couldn`t they put them on the releases we already bought",negative,75,"sons of ****, why couldn`t they put them on the releases we already bought","[sons, of, *, *, *, *, ,, why, couldn, `, t, they, put, them, on, the, releases, we, already, bought]","[sons, *, *, *, *, ,, `, put, releases, already, bought]","[sons, put, releases, already, bought]"
...,...,...,...,...,...,...,...
27476,wish we could come see u on Denver husband lost his job and can`t afford it,negative,77,wish we could come see u on denver husband lost his job and can`t afford it,"[wish, we, could, come, see, u, on, denver, husband, lost, his, job, and, can, `, t, afford, it]","[wish, could, come, see, u, denver, husband, lost, job, `, afford]","[wish, could, come, see, u, denver, husband, lost, job, afford]"
27477,"I`ve wondered about rake to. The client has made it clear .NET only, don`t force devs to learn a new lang #agile #ccnet",negative,122,"i`ve wondered about rake to. the client has made it clear .net only, don`t force devs to learn a new lang #agile #ccnet","[i, `, ve, wondered, about, rake, to, ., the, client, has, made, it, clear, .net, only, ,, don, `, t, force, devs, to, learn, a, new, lang, #, agile, #, ccnet]","[`, wondered, rake, ., client, made, clear, .net, ,, `, force, devs, learn, new, lang, #, agile, #, ccnet]","[wondered, rake, client, made, clear, .net, force, devs, learn, new, lang, agile, ccnet]"
27478,Yay good for both of you. Enjoy the break - you probably need it after such hectic weekend Take care hun xxxx,positive,111,yay good for both of you. enjoy the break - you probably need it after such hectic weekend take care hun xxxx,"[yay, good, for, both, of, you, ., enjoy, the, break, -, you, probably, need, it, after, such, hectic, weekend, take, care, hun, xxxx]","[yay, good, ., enjoy, break, -, probably, need, hectic, weekend, take, care, hun, xxxx]","[yay, good, enjoy, break, probably, need, hectic, weekend, take, care, hun, xxxx]"
27479,But it was worth it ****.,positive,27,but it was worth it ****.,"[but, it, was, worth, it, *, *, *, *, .]","[worth, *, *, *, *, .]",[worth]


In [24]:
## [v3 For Loop - Continue] Define function to remove URLs
def remove_urls(token_list):
    no_urls = []
    for token in token_list:
        if ('http' in token) | ('www' not in token):
            continue
        no_urls.append(token)
    return no_urls

## Remove URLs from no_stops_no_punct
df['no_stops_no_punct'] = df['no_stops_no_punct'].apply(remove_urls)
df

,text,sentiment,length,lower_text,tokens,no_stops,no_stops_no_punct
0,"I`d have responded, if I were going",neutral,36,"i`d have responded, if i were going","[i, `, d, have, responded, ,, if, i, were, going]","[`, responded, ,, going]",[]
1,Sooo SAD I will miss you here in San Diego!!!,negative,46,sooo sad i will miss you here in san diego!!!,"[sooo, sad, i, will, miss, you, here, in, san, diego, !, !, !]","[sooo, sad, miss, san, diego, !, !, !]",[]
2,my boss is bullying me...,negative,25,my boss is bullying me...,"[my, boss, is, bullying, me, ...]","[boss, bullying, ...]",[]
3,what interview! leave me alone,negative,31,what interview! leave me alone,"[what, interview, !, leave, me, alone]","[interview, !, leave, alone]",[]
4,"Sons of ****, why couldn`t they put them on the releases we already bought",negative,75,"sons of ****, why couldn`t they put them on the releases we already bought","[sons, of, *, *, *, *, ,, why, couldn, `, t, they, put, them, on, the, releases, we, already, bought]","[sons, *, *, *, *, ,, `, put, releases, already, bought]",[]
...,...,...,...,...,...,...,...
27476,wish we could come see u on Denver husband lost his job and can`t afford it,negative,77,wish we could come see u on denver husband lost his job and can`t afford it,"[wish, we, could, come, see, u, on, denver, husband, lost, his, job, and, can, `, t, afford, it]","[wish, could, come, see, u, denver, husband, lost, job, `, afford]",[]
27477,"I`ve wondered about rake to. The client has made it clear .NET only, don`t force devs to learn a new lang #agile #ccnet",negative,122,"i`ve wondered about rake to. the client has made it clear .net only, don`t force devs to learn a new lang #agile #ccnet","[i, `, ve, wondered, about, rake, to, ., the, client, has, made, it, clear, .net, only, ,, don, `, t, force, devs, to, learn, a, new, lang, #, agile, #, ccnet]","[`, wondered, rake, ., client, made, clear, .net, ,, `, force, devs, learn, new, lang, #, agile, #, ccnet]",[]
27478,Yay good for both of you. Enjoy the break - you probably need it after such hectic weekend Take care hun xxxx,positive,111,yay good for both of you. enjoy the break - you probably need it after such hectic weekend take care hun xxxx,"[yay, good, for, both, of, you, ., enjoy, the, break, -, you, probably, need, it, after, such, hectic, weekend, take, care, hun, xxxx]","[yay, good, ., enjoy, break, -, probably, need, hectic, weekend, take, care, hun, xxxx]",[]
27479,But it was worth it ****.,positive,27,but it was worth it ****.,"[but, it, was, worth, it, *, *, *, *, .]","[worth, *, *, *, *, .]",[]


In [25]:
df.head(10)

,text,sentiment,length,lower_text,tokens,no_stops,no_stops_no_punct
0,"I`d have responded, if I were going",neutral,36,"i`d have responded, if i were going","[i, `, d, have, responded, ,, if, i, were, going]","[`, responded, ,, going]",[]
1,Sooo SAD I will miss you here in San Diego!!!,negative,46,sooo sad i will miss you here in san diego!!!,"[sooo, sad, i, will, miss, you, here, in, san, diego, !, !, !]","[sooo, sad, miss, san, diego, !, !, !]",[]
2,my boss is bullying me...,negative,25,my boss is bullying me...,"[my, boss, is, bullying, me, ...]","[boss, bullying, ...]",[]
3,what interview! leave me alone,negative,31,what interview! leave me alone,"[what, interview, !, leave, me, alone]","[interview, !, leave, alone]",[]
4,"Sons of ****, why couldn`t they put them on the releases we already bought",negative,75,"sons of ****, why couldn`t they put them on the releases we already bought","[sons, of, *, *, *, *, ,, why, couldn, `, t, they, put, them, on, the, releases, we, already, bought]","[sons, *, *, *, *, ,, `, put, releases, already, bought]",[]
5,http://www.dothebouncy.com/smf - some shameless plugging for the best Rangers forum on earth,neutral,92,http://www.dothebouncy.com/smf - some shameless plugging for the best rangers forum on earth,"[http, :, //www.dothebouncy.com/smf, -, some, shameless, plugging, for, the, best, rangers, forum, on, earth]","[http, :, //www.dothebouncy.com/smf, -, shameless, plugging, best, rangers, forum, earth]",[]
6,2am feedings for the baby are fun when he is all smiles and coos,positive,64,2am feedings for the baby are fun when he is all smiles and coos,"[2am, feedings, for, the, baby, are, fun, when, he, is, all, smiles, and, coos]","[2am, feedings, baby, fun, smiles, coos]",[]
7,Soooo high,neutral,10,soooo high,"[soooo, high]","[soooo, high]",[]
8,Both of you,neutral,12,both of you,"[both, of, you]",[],[]
9,Journey!? Wow... u just became cooler. hehe... (is that possible!?),positive,69,journey!? wow... u just became cooler. hehe... (is that possible!?),"[journey, !, ?, wow, ..., u, just, became, cooler, ., hehe, ..., (, is, that, possible, !, ?, )]","[journey, !, ?, wow, ..., u, became, cooler, ., hehe, ..., (, possible, !, ?, )]",[]


## Results

Note how many fewer tokens we have in our `no_stops_no_punct` tokens than in our original.  However, some information was lost, but a lot was also retained.  

Normalization is a huge part of the NLP process and is always a balance between reducing the size of our vocabulary and therefor simplifying our models, and retaining enough information for the model to extract some meaningful patterns in the texts.  

There are a lot of choices here to make.

# Normalizing Text with spaCy

The spaCy Python package provides text processing pipelines that can do many of these operations, plus much more complicated processing, very fast and in many fewer steps.  For this reason it is a very popular tool.  

It utilizes pretrained language models that can recognize things like parts of speech and named entities (people, specific places, currency, etc.)

spaCy was not included in your original dojo_env, so you will need to install if if you have not already.

We will also download the pretrained english language model trained on millions of web documents.  We will use the small sized one for efficiency.

In [26]:
## Install spacy if necessary
# (best if run in terminal without the "%" first character)
# %conda install -c conda-forge spacy

In [27]:
import spacy

## Download the English small-sized model trained on web documents if necessary
# spacy.cli.download('en_core_web_sm')

## The spaCy model

In [28]:
## Load the model.  Disable Named Entity Recognizer (too slow)
nlp_model = spacy.load('en_core_web_sm', disable='ner')
nlp_model.pipe_names

['tok2vec', 'tagger', 'parser', 'attribute_ruler', 'lemmatizer']

We have our model, and we can apply it like a function.  It expects a string of text as the input.

In [29]:
## Process a document with the model
doc = nlp_model(df['text'][5])
doc

http://www.dothebouncy.com/smf - some shameless plugging for the best Rangers forum on earth

The document is a collection of tokens we can iterate over

## Documents and Tokens

In [30]:
## Display the tokens in the document
[token for token in doc]

[http://www.dothebouncy.com/smf,
 -,
 some,
 shameless,
 plugging,
 for,
 the,
 best,
 Rangers,
 forum,
 on,
 earth]

Each token is much more than a string.  It

In [31]:
## Isolate the last token in the document
word = doc[-1]

## Display the text and type of the token
print(word)
type(word)

earth


spacy.tokens.token.Token

Each has many attributes that we can take advantage of, such as the lemma form and whether it is punctuation or space, and whether it is a stop word

In [32]:
## Display the lemmatized form of the token
word.lemma_

'earth'

In [33]:
## Check whether the token is punctuation
word.is_punct

False

In [34]:
## Check whether the token is a space
word.is_space

False

Spacy can even determine the part of speech that the token is!

In [35]:
## Check the part of speech of the token
word.pos_

'NOUN'

In [36]:
[token.pos_ for token in doc]

['PROPN',
 'PUNCT',
 'DET',
 'ADJ',
 'VERB',
 'ADP',
 'DET',
 'ADJ',
 'PROPN',
 'NOUN',
 'ADP',
 'NOUN']

In [37]:
## [v1: List Comp] Make a list of the lemmas for each token in the document
[token.lemma_ for token in doc]

['http://www.dothebouncy.com/smf',
 '-',
 'some',
 'shameless',
 'plug',
 'for',
 'the',
 'good',
 'Rangers',
 'forum',
 'on',
 'earth']

In [38]:
## [v2: For Loop] Make a list of the lemmas for each token in the document- '
lemmas_list = []
for token in doc:
    lemmas_list.append( token.lemma_)
lemmas_list

['http://www.dothebouncy.com/smf',
 '-',
 'some',
 'shameless',
 'plug',
 'for',
 'the',
 'good',
 'Rangers',
 'forum',
 'on',
 'earth']

Notice that the spaCy lemmatization does not automatically lower the casing of words when lemmatizing.  Let's go ahead and make sure they are all lower case.

In [39]:
## [v1 List Comp] Make a list of only the tokens in the document that are not punctuation or spaces
## Lower the casing as well
[token.lemma_.lower() for token in doc if not token.is_punct and not token.is_space]

['http://www.dothebouncy.com/smf',
 'some',
 'shameless',
 'plug',
 'for',
 'the',
 'good',
 'rangers',
 'forum',
 'on',
 'earth']

In [40]:
## [v2 For Loop - Continue] Make a list of only the tokens in the document that are not punctuation or spaces 
## Lower the casing as well
lemmas_list = []
for token in doc:
    if token.is_punct:
        continue
    if token.is_space:
        continue

    lemmas_list.append(token.lemma_.lower())

lemmas_list

['http://www.dothebouncy.com/smf',
 'some',
 'shameless',
 'plug',
 'for',
 'the',
 'good',
 'rangers',
 'forum',
 'on',
 'earth']

In [41]:
## [v1 List Comp] Make a list of all the tokens in the document that are not punctuation, spaces, or stop words
[token.lemma_.lower() for token in doc if not token.is_punct and not token.is_space and not token.is_stop]

['http://www.dothebouncy.com/smf',
 'shameless',
 'plug',
 'good',
 'rangers',
 'forum',
 'earth']

In [42]:
## [v3 For Loop - Continue] Make a list of only the tokens in the document that are not punctuation or spaces 
## Lower the casing as well
lemmas_list = []
for token in doc:
    if token.is_punct:
        continue
    if token.is_space:
        continue
    if token.is_stop:
        continue

    lemmas_list.append(token.lemma_.lower())

lemmas_list

['http://www.dothebouncy.com/smf',
 'shameless',
 'plug',
 'good',
 'rangers',
 'forum',
 'earth']

In [43]:
## [v1 List Comp] Let's also remove the url
[token.lemma_.lower() for token in doc if not token.is_punct and 
 not token.is_space and not token.is_stop and 
 not 'http' in token.lemma_.lower() and 'www' not in token.lemma_.lower()]

['shameless', 'plug', 'good', 'rangers', 'forum', 'earth']

In [44]:
## [v2 For Loop] Let's also remove the url
## Lower the casing as well
lemmas_list = []
for token in doc:
    if token.is_punct:
        continue
    if token.is_space:
        continue
    if token.is_stop:
        continue

    if 'http' in token.text.lower():
        continue
    if 'www' in token.text.lower():
        continue
        
    lemmas_list.append(token.lemma_.lower())

lemmas_list

['shameless', 'plug', 'good', 'rangers', 'forum', 'earth']

In order to use spaCy to process our entire dataframe, we will need to make a function and apply it to our text column.

## Preprocessing with spaCy

In [45]:
## [V1 List Comp] Define a function to use spacy to process our text
def spacy_process(text):
        """Lemmatize tokens, lower case, remove punctuation, spaces, and stop words"""
        doc = nlp_model(text)
        processed_doc = [token.lemma_.lower() for token in doc if not token.is_punct and 
                         not token.is_space and not token.is_stop and 
                         not 'http' in token.lemma_.lower() and 'www' not in token.lemma_.lower()]
        return processed_doc

## process the tweets using the spacy function
df['spacy_lemmas'] = df['text'].apply(spacy_process)
df.head()

,text,sentiment,length,lower_text,tokens,no_stops,no_stops_no_punct,spacy_lemmas
0,"I`d have responded, if I were going",neutral,36,"i`d have responded, if i were going","[i, `, d, have, responded, ,, if, i, were, going]","[`, responded, ,, going]",[],"[i`d, respond, go]"
1,Sooo SAD I will miss you here in San Diego!!!,negative,46,sooo sad i will miss you here in san diego!!!,"[sooo, sad, i, will, miss, you, here, in, san, diego, !, !, !]","[sooo, sad, miss, san, diego, !, !, !]",[],"[sooo, sad, miss, san, diego]"
2,my boss is bullying me...,negative,25,my boss is bullying me...,"[my, boss, is, bullying, me, ...]","[boss, bullying, ...]",[],"[boss, bully]"
3,what interview! leave me alone,negative,31,what interview! leave me alone,"[what, interview, !, leave, me, alone]","[interview, !, leave, alone]",[],"[interview, leave]"
4,"Sons of ****, why couldn`t they put them on the releases we already bought",negative,75,"sons of ****, why couldn`t they put them on the releases we already bought","[sons, of, *, *, *, *, ,, why, couldn, `, t, they, put, them, on, the, releases, we, already, bought]","[sons, *, *, *, *, ,, `, put, releases, already, bought]",[],"[son, couldn`t, release, buy]"


In [46]:
# [v2: for Loop] Define a function to use spacy to process our text
def spacy_process(text):
    """Lemmatize tokens, lower case, remove punctuation, spaces, and stop words"""

    # Make text into a document
    doc = nlp_model(text)
    
    processed_doc = [ ]
    
    for token in doc:
        # Check if token is stopword
        if token.is_stop == True:
            # Continue the loop with the next token
            continue
    
        # Check if  is punct
        if (remove_punct == True) and (token.is_punct == True):
            continue
    
        # Check if  is whitespace
        if (remove_punct == True) and (token.is_space == True):
            continue

        ## Check if URL
        if (remove_urls==True) & ('http' in token.text.lower()):
            continue
            
        ## Check if URL
        if (remove_urls==True) & ('www' in token.text.lower()):
            continue
    
        ## Determine final form of output list of tokens/lemmas
        processed_doc.append(token.lemma_.lower())
    
    return processed_doc



## process the tweets using the spacy function
df['spacy_lemmas'] = df['text'].apply(spacy_process)
df.head()


,text,sentiment,length,lower_text,tokens,no_stops,no_stops_no_punct,spacy_lemmas
0,"I`d have responded, if I were going",neutral,36,"i`d have responded, if i were going","[i, `, d, have, responded, ,, if, i, were, going]","[`, responded, ,, going]",[],"[ , i`d, respond, ,, go]"
1,Sooo SAD I will miss you here in San Diego!!!,negative,46,sooo sad i will miss you here in san diego!!!,"[sooo, sad, i, will, miss, you, here, in, san, diego, !, !, !]","[sooo, sad, miss, san, diego, !, !, !]",[],"[ , sooo, sad, miss, san, diego, !, !, !]"
2,my boss is bullying me...,negative,25,my boss is bullying me...,"[my, boss, is, bullying, me, ...]","[boss, bullying, ...]",[],"[boss, bully, ...]"
3,what interview! leave me alone,negative,31,what interview! leave me alone,"[what, interview, !, leave, me, alone]","[interview, !, leave, alone]",[],"[ , interview, !, leave]"
4,"Sons of ****, why couldn`t they put them on the releases we already bought",negative,75,"sons of ****, why couldn`t they put them on the releases we already bought","[sons, of, *, *, *, *, ,, why, couldn, `, t, they, put, them, on, the, releases, we, already, bought]","[sons, *, *, *, *, ,, `, put, releases, already, bought]",[],"[ , son, *, *, *, *, ,, couldn`t, release, buy]"


We used spaCy to tokenize, lemmatize, and remove punctuation and stopwords from our text in one step!

Notice that the spaCy processed data is a little different than our previously processed data.  The text has been lemmatized and spaCy has a different list of stop words than NLTK.

The learn platform has directions for how you can customize your spaCy stopword list and a function with more flexibility in how spaCy will process your data.

# Ngrams

ngrams combine multiple words into tokens

In [47]:
## Import the ngrams function
from nltk import ngrams

In [48]:
## Isolate the first lemmatized document
lemma_doc = df['spacy_lemmas'][5]
lemma_doc

['http://www.dothebouncy.com/smf',
 '-',
 'shameless',
 'plug',
 'good',
 'rangers',
 'forum',
 'earth']

In [49]:
# Create bigrams
list(ngrams(lemma_doc,2))

[('http://www.dothebouncy.com/smf', '-'),
 ('-', 'shameless'),
 ('shameless', 'plug'),
 ('plug', 'good'),
 ('good', 'rangers'),
 ('rangers', 'forum'),
 ('forum', 'earth')]

In [50]:
# Create trigrams
list(ngrams(lemma_doc,3))

[('http://www.dothebouncy.com/smf', '-', 'shameless'),
 ('-', 'shameless', 'plug'),
 ('shameless', 'plug', 'good'),
 ('plug', 'good', 'rangers'),
 ('good', 'rangers', 'forum'),
 ('rangers', 'forum', 'earth')]

## Applying `ngrams` to make a new column

We need to make a function that returns a list of bigrams.  It won't work to just pass the ngrams function to `.apply()`

In [51]:
## Create a function to create bigrams
def make_bigrams(doc):
    bigrams = ngrams(doc, 2)
    bigrams = list(bigrams)
    return bigrams

In [52]:
# add bigrams to the df with .apply()
df['bigrams'] = df['spacy_lemmas'].apply(make_bigrams)
df.head()

,text,sentiment,length,lower_text,tokens,no_stops,no_stops_no_punct,spacy_lemmas,bigrams
0,"I`d have responded, if I were going",neutral,36,"i`d have responded, if i were going","[i, `, d, have, responded, ,, if, i, were, going]","[`, responded, ,, going]",[],"[ , i`d, respond, ,, go]","[( , i`d), (i`d, respond), (respond, ,), (,, go)]"
1,Sooo SAD I will miss you here in San Diego!!!,negative,46,sooo sad i will miss you here in san diego!!!,"[sooo, sad, i, will, miss, you, here, in, san, diego, !, !, !]","[sooo, sad, miss, san, diego, !, !, !]",[],"[ , sooo, sad, miss, san, diego, !, !, !]","[( , sooo), (sooo, sad), (sad, miss), (miss, san), (san, diego), (diego, !), (!, !), (!, !)]"
2,my boss is bullying me...,negative,25,my boss is bullying me...,"[my, boss, is, bullying, me, ...]","[boss, bullying, ...]",[],"[boss, bully, ...]","[(boss, bully), (bully, ...)]"
3,what interview! leave me alone,negative,31,what interview! leave me alone,"[what, interview, !, leave, me, alone]","[interview, !, leave, alone]",[],"[ , interview, !, leave]","[( , interview), (interview, !), (!, leave)]"
4,"Sons of ****, why couldn`t they put them on the releases we already bought",negative,75,"sons of ****, why couldn`t they put them on the releases we already bought","[sons, of, *, *, *, *, ,, why, couldn, `, t, they, put, them, on, the, releases, we, already, bought]","[sons, *, *, *, *, ,, `, put, releases, already, bought]",[],"[ , son, *, *, *, *, ,, couldn`t, release, buy]","[( , son), (son, *), (*, *), (*, *), (*, *), (*, ,), (,, couldn`t), (couldn`t, release), (release, buy)]"


In [53]:
df.head(10)

,text,sentiment,length,lower_text,tokens,no_stops,no_stops_no_punct,spacy_lemmas,bigrams
0,"I`d have responded, if I were going",neutral,36,"i`d have responded, if i were going","[i, `, d, have, responded, ,, if, i, were, going]","[`, responded, ,, going]",[],"[ , i`d, respond, ,, go]","[( , i`d), (i`d, respond), (respond, ,), (,, go)]"
1,Sooo SAD I will miss you here in San Diego!!!,negative,46,sooo sad i will miss you here in san diego!!!,"[sooo, sad, i, will, miss, you, here, in, san, diego, !, !, !]","[sooo, sad, miss, san, diego, !, !, !]",[],"[ , sooo, sad, miss, san, diego, !, !, !]","[( , sooo), (sooo, sad), (sad, miss), (miss, san), (san, diego), (diego, !), (!, !), (!, !)]"
2,my boss is bullying me...,negative,25,my boss is bullying me...,"[my, boss, is, bullying, me, ...]","[boss, bullying, ...]",[],"[boss, bully, ...]","[(boss, bully), (bully, ...)]"
3,what interview! leave me alone,negative,31,what interview! leave me alone,"[what, interview, !, leave, me, alone]","[interview, !, leave, alone]",[],"[ , interview, !, leave]","[( , interview), (interview, !), (!, leave)]"
4,"Sons of ****, why couldn`t they put them on the releases we already bought",negative,75,"sons of ****, why couldn`t they put them on the releases we already bought","[sons, of, *, *, *, *, ,, why, couldn, `, t, they, put, them, on, the, releases, we, already, bought]","[sons, *, *, *, *, ,, `, put, releases, already, bought]",[],"[ , son, *, *, *, *, ,, couldn`t, release, buy]","[( , son), (son, *), (*, *), (*, *), (*, *), (*, ,), (,, couldn`t), (couldn`t, release), (release, buy)]"
5,http://www.dothebouncy.com/smf - some shameless plugging for the best Rangers forum on earth,neutral,92,http://www.dothebouncy.com/smf - some shameless plugging for the best rangers forum on earth,"[http, :, //www.dothebouncy.com/smf, -, some, shameless, plugging, for, the, best, rangers, forum, on, earth]","[http, :, //www.dothebouncy.com/smf, -, shameless, plugging, best, rangers, forum, earth]",[],"[http://www.dothebouncy.com/smf, -, shameless, plug, good, rangers, forum, earth]","[(http://www.dothebouncy.com/smf, -), (-, shameless), (shameless, plug), (plug, good), (good, rangers), (rangers, forum), (forum, earth)]"
6,2am feedings for the baby are fun when he is all smiles and coos,positive,64,2am feedings for the baby are fun when he is all smiles and coos,"[2am, feedings, for, the, baby, are, fun, when, he, is, all, smiles, and, coos]","[2am, feedings, baby, fun, smiles, coos]",[],"[2, feeding, baby, fun, smile, coo]","[(2, feeding), (feeding, baby), (baby, fun), (fun, smile), (smile, coo)]"
7,Soooo high,neutral,10,soooo high,"[soooo, high]","[soooo, high]",[],"[soooo, high]","[(soooo, high)]"
8,Both of you,neutral,12,both of you,"[both, of, you]",[],[],[ ],[]
9,Journey!? Wow... u just became cooler. hehe... (is that possible!?),positive,69,journey!? wow... u just became cooler. hehe... (is that possible!?),"[journey, !, ?, wow, ..., u, just, became, cooler, ., hehe, ..., (, is, that, possible, !, ?, )]","[journey, !, ?, wow, ..., u, became, cooler, ., hehe, ..., (, possible, !, ?, )]",[],"[ , journey, !, ?, wow, ..., u, cool, ., , hehe, ..., (, possible, !, ?, )]","[( , journey), (journey, !), (!, ?), (?, wow), (wow, ...), (..., u), (u, cool), (cool, .), (., ), ( , hehe), (hehe, ...), (..., (), ((, possible), (possible, !), (!, ?), (?, ))]"


# Save the final data version for modeling

In [54]:
## Save the processed data
df.to_csv('../Data/processed_data.csv', index=False)

In [55]:
## Save the processed data - as a joblib
import joblib
joblib.dump(df,'../Data/processed_data.joblib')


['../Data/processed_data.joblib']